# Fairness Metrics. 


---


## TODO:

  * For the selected data set, train at least one tree-based ensemble model (random forest, gbm, catboost or any other boosting)
  * For a selected variable (protected attribute, if possible choose sex or race, but if you do not have such variable then choose any categorical varaible) calculate selected fairness metrics (at least 3, like statistical parity, • equal opportunity, predictive equality),
  * train three or more candidate models (different variables, different transformations, different model structures) and compare fairness metrics between them.
  * Comment on the results for points (2) and (3)


## Data & preprocessing


---



For the sake of simplicity we'll re-use mushrooms dataset again, known already from the HW1, HW2 and HW3. Ordinnal encoder will be used, similarly to the HW3.

In [22]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("mushrooms.csv")
encoder = OrdinalEncoder()

y = df.loc[:,['class']]
X = df.drop(['class'], axis=1)

y_temp = encoder.fit_transform(y)
X_temp = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

And we'll also need fairness metrics.

In [23]:
!pip install --quiet 'aif360[all]'
from aif360.sklearn.metrics import statistical_parity_difference, equal_opportunity_difference, between_group_generalized_entropy_error

## First model

---

For the tree-based model, let's take xgboost.

In [24]:
import xgboost

xgb = xgboost.XGBClassifier().fit(X_train, y_train)
print(f"Model score on test set: {xgb.score(X_test, y_test)}")
print(f"Predictions for the first 10 observations: {xgb.predict(X_test[:10])}")
print(f"True values for the first 10 observations: {y_test[:10].tolist()}")

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


Model score on test set: 1.0
Predictions for the first 10 observations: [1. 1. 0. 1. 0. 1. 0. 0. 0. 0.]
True values for the first 10 observations: [[1.0], [1.0], [0.0], [1.0], [0.0], [1.0], [0.0], [0.0], [0.0], [0.0]]


Since mushroom dataset is fairly simple, our xboost model trained very well. Let's calculate fairness metrics and see how they behave.

In [51]:
import pandas as pd

y_true = pd.DataFrame(y_test)
#y_true = y_true.reset_index()
y_true.columns = ['Poisonous']
#y_pred = pd.DataFrame(xgb.predict(X_test), columns='Poisonous')
#statistical_parity_difference(predictions)

In [52]:
y_true

,Poisonous
0,1.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
1620,1.0
1621,1.0
1622,1.0
1623,1.0


In [55]:
equal_opportunity_difference(y_true, y_pred, priv_group='Poisonous')

ValueError: ignored

## Second model

---

In [ ]:
# And now, for the last point, let's re-use some code from HW1 to train simple NN classifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

model = Sequential()
#model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=16, epochs=100, validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
325/325 [==============================] - 1s 2ms/step - loss: 0.9309 - accuracy: 0.5549 - val_loss: 0.8970 - val_accuracy: 0.5685
Epoch 2/100
325/325 [==============================] - 0s 2ms/step - loss: 0.8409 - accuracy: 0.5807 - val_loss: 0.8647 - val_accuracy: 0.5754
Epoch 3/100
325/325 [==============================] - 0s 1ms/step - loss: 0.8260 - accuracy: 0.5890 - val_loss: 0.8331 - val_accuracy: 0.5977
Epoch 4/100
325/325 [==============================] - 0s 1ms/step - loss: 0.7907 - accuracy: 0.6159 - val_loss: 0.8040 - val_accuracy: 0.6223
Epoch 5/100
325/325 [==============================] - 0s 1ms/step - loss: 0.7636 - accuracy: 0.6414 - val_loss: 0.7779 - val_accuracy: 0.6338
Epoch 6/100
325/325 [==============================] - 0s 2ms/step - loss: 0.7446 - accuracy: 0.6480 - val_loss: 0.7530 - val_accuracy: 0.6485
Epoch 7/100
325/325 [==============================] - 0s 1ms/step - loss: 0.7154 - accuracy: 0.6654 - val_loss: 0.7305 - val_accuracy: 0.6623

## Third model

---

In [ ]:
# And now, for the last point, let's re-use some code from HW1 to train simple NN classifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

model_2 = Sequential()
model_2.add(Dense(256, activation='relu'))Y
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(32, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(1, activation='sigmoid')) 

model_2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
history = model_2.fit(X_train, y_train, batch_size=16, epochs=30, validation_data=(X_val, y_val))

score = model_2.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
325/325 [==============================] - 2s 4ms/step - loss: 0.6777 - accuracy: 0.5895 - val_loss: 0.4041 - val_accuracy: 0.8438
Epoch 2/30
325/325 [==============================] - 1s 3ms/step - loss: 0.4669 - accuracy: 0.7913 - val_loss: 0.2960 - val_accuracy: 0.8915
Epoch 3/30
325/325 [==============================] - 1s 3ms/step - loss: 0.3828 - accuracy: 0.8470 - val_loss: 0.2295 - val_accuracy: 0.9138
Epoch 4/30
325/325 [==============================] - 1s 3ms/step - loss: 0.3113 - accuracy: 0.8737 - val_loss: 0.1863 - val_accuracy: 0.9415
Epoch 5/30
325/325 [==============================] - 1s 3ms/step - loss: 0.2696 - accuracy: 0.8968 - val_loss: 0.1626 - val_accuracy: 0.9400
Epoch 6/30
325/325 [==============================] - 1s 3ms/step - loss: 0.2433 - accuracy: 0.9030 - val_loss: 0.1438 - val_accuracy: 0.9462
Epoch 7/30
325/325 [==============================] - 1s 3ms/step - loss: 0.2033 - accuracy: 0.9264 - val_loss: 0.1254 - val_accuracy: 0.9469
Epoch 